- 한빛 아카데미
    - https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1      (메인)
    - https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B7192437175      (서브)
    location='/academy/books/book_view.html?p_code=B7192437175' (알 수 있는 정보)
    - 제목, 저자, 출간인, 쪽수, 가격, 역자
    

In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

#### 1. 데이터 가져오기

In [2]:
base_url = 'https://www.hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
result = requests.get(url)
soup = BeautifulSoup(result.text)

#### 2. 찾으려고 하는 데이터 찾기

In [3]:
lis = soup.select('li.sub_book_list')
len(lis)

20

#### 3. 하나의 데이터로 원하는 정보 추출

In [9]:
li = lis[0]
href = li.select_one('.pop_quick_bg')['onclick']
href

"location='/academy/books/book_view.html?p_code=B7192437175'"

In [10]:
href = href[10:-1]
href

'/academy/books/book_view.html?p_code=B7192437175'

- subpage에서 정보 추출하기 

In [12]:
sub_url = f'{base_url}{href}'
res = requests.get(sub_url)
book_soup = BeautifulSoup(res.text)

In [13]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').text.strip()
title

'처음 배우는 매트랩'

In [22]:
# 저자
book_lis = info.select('.info_list > li')
author = ','.join(book_lis[0].find('span').get_text().strip().split(' , '))
author

'방성완'

In [23]:
# 역자
if len(book_lis) == 5:
    translator = '-'
else:
    translator = ','.join(book_lis[1].find('span').get_text().strip().split(' , '))
translator

'-'

In [25]:
# 출간일
date = book_lis[-4].find('span').get_text()
date

'2023-04-10'

In [30]:
# 페이지
page = int(book_lis[-3].find('span').get_text().strip()[:-2])
page

416

In [32]:
# 가격
payment = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payment[1].select_one('.pbr').find('strong').get_text().strip()
sales

'30,000'

In [35]:
##### 4. 한 페이지에 있는 데이터
lines = []
for li in lis:
    href = li.select_one('.pop_quick_bg')['onclick'][10:-1]
    res = requests.get(base_url + href)
    book_soup = BeautifulSoup(res.text)
    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').text.strip()
    book_lis = info.select('.info_list > li')
    author = ','.join(book_lis[0].find('span').get_text().strip().split(' , '))
    if len(book_lis) == 5:
        translator = '-'
    else:
        translator = ','.join(book_lis[1].find('span').get_text().strip().split(' , '))
    date = book_lis[-4].find('span').get_text()
    page = int(book_lis[-3].find('span').get_text().strip()[:-2])
    payment = book_soup.select_one('.payment_box.curr').find_all('p')
    sales = payment[1].select_one('.pbr').find('strong').get_text().strip()
    lines.append({
        '제목':title, '저자':author, '역자':translator,
        '출간일':date, '페이지수':page, '판매가':sales
    })

In [36]:
df1 = pd.DataFrame(lines)
df1

,제목,저자,역자,출간일,페이지수,판매가
0,처음 배우는 매트랩,방성완,-,2023-04-10,416,"30,000"
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,-,2023-03-09,516,"34,000"
2,정보교과교육론(3판),"최현종,전용주",-,2023-03-01,488,"30,000"
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,-,2023-02-28,308,"22,000"
4,경제수학 강의(3판),김성현,-,2023-02-28,464,"23,000"
5,글로벌 사회와 리더십(2판),안광현,-,2023-02-28,416,"28,000"
6,행동 재무론,고광수,-,2023-02-28,596,"32,000"
7,"IT CookBook, 알기 쉽게 해설한 파이썬 with 컴퓨팅 사고력","김충석,김지연",-,2023-02-24,328,"24,000"
8,MATLAB으로 배우는 공학 수치해석(3판),방성완,2023-02-20,360 쪽,97911566463,"28,000"
9,Cengel의 알기 쉬운 열역학(6판),"Yunus A. Çengel,John M. Cimbala,Afshin J. Ghajar","김택영,한진호,김덕종,김우태,이무연,전문수",2023-02-17,528,"34,000"
